In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install Wandb for monitoring
!pip install wandb

In [ ]:
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
# Login to Hugging Face
HUGGINGFACE_TOKEN = ""
from huggingface_hub import login
login(token = HUGGINGFACE_TOKEN)

In [ ]:
# Login to Wandb
# Get API key from https://wandb.ai/settings#api
WANDB_API_KEY = ""
import wandb
wandb.login(key = WANDB_API_KEY)
run = wandb.init(project = "Mistral-finetuned-path-generation")

In [ ]:
pip install --upgrade torch torchvision

In [ ]:
!pip install --upgrade unsloth torch torchvision transformers accelerate bitsandbytes

In [ ]:
!pip install unsloth
!pip install "trl<0.15.0"

In [ ]:
import torchvision
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None # Auto detection of dtype
load_in_4bit = True # Reduce memory usage with 4-bit quantization
model, tokenizer = FastLanguageModel.from_pretrained(
 model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
 max_seq_length = max_seq_length,
 dtype = dtype,
 load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Mistral patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
 model,
 r = 16, # LoRA rank
 target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
 lora_alpha = 16,
 lora_dropout = 0, # Optimized at 0
 bias = "none", # No additional bias terms
 use_gradient_checkpointing = "unsloth", # Gradient checkpointing to save memory
 random_state = 3407,
 use_rslora = False, # Rank stabilized LoRA, can be enabled for stability
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

dataset = load_dataset('matteanedda/path_selection_bidirectionality_and_connection_understanding_master', split="train")

README.md:   0%|          | 0.00/355 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [ ]:
dataset[5]["conversations"]

[{'from': 'human',
  'value': 'Validate these itineraries respecting the following rules:\n\n[RULES]\n1. START: Starts with a non-transport location\n2. CONTINUITY: Each stop must connect to the next\n3. TRANSPORT END: Every itinerary MUST end at a transport location\n4. NO INTERMEDIATE TRANSPORT: Transport locations only allowed as final stop\n5. LENGTH: Itineraries can have 3-6 stops\n6. UNIQUE: No location can be used twice in the same itinerary\n7. DISTINCT: Intermediate stops should be different across itineraries\n8. BIDIRECTION: The connections are bidirectionals\n9. WALKING TIME: Walking time must be retrieved from the connection list\n[BIDIRECTIONAL CONNECTIONS - IMPORTANT]\nEvery connection in this system works both ways:\n1. If you can walk from A → B in X minutes\n2. You can ALWAYS walk from B → A in the same X minutes\n3. Example from the connections list:\n   - If "Museum: Louvre → Church: Notre Dame = 10 min" exists\n   - Then "Church: Notre Dame → Museum: Louvre = 10 mi

In [ ]:
print(dataset[5]["text"])

<|im_start|>user
Validate these itineraries respecting the following rules:

[RULES]
1. START: Starts with a non-transport location
2. CONTINUITY: Each stop must connect to the next
3. TRANSPORT END: Every itinerary MUST end at a transport location
4. NO INTERMEDIATE TRANSPORT: Transport locations only allowed as final stop
5. LENGTH: Itineraries can have 3-6 stops
6. UNIQUE: No location can be used twice in the same itinerary
7. DISTINCT: Intermediate stops should be different across itineraries
8. BIDIRECTION: The connections are bidirectionals
9. WALKING TIME: Walking time must be retrieved from the connection list
[BIDIRECTIONAL CONNECTIONS - IMPORTANT]
Every connection in this system works both ways:
1. If you can walk from A → B in X minutes
2. You can ALWAYS walk from B → A in the same X minutes
3. Example from the connections list:
   - If "Museum: Louvre → Church: Notre Dame = 10 min" exists
   - Then "Church: Notre Dame → Museum: Louvre = 10 min" is automatically valid
4. Key

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 350,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        # optim = "adamw_8bit"
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 350
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,0.793100
2,0.741400
3,0.597000
4,0.681200
5,0.657500
6,0.456700
7,0.565000
8,0.469500
9,0.366100
10,0.337200


Step,Training Loss
1,0.793100
2,0.741400
3,0.597000
4,0.681200
5,0.657500
6,0.456700
7,0.565000
8,0.469500
9,0.366100
10,0.337200


In [ ]:
run.finish()

train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇█████
train/grad_norm,▅▆▅██▃▃▂▃▁▂▇▂▁▁▁▂▁▁▁▁▂▁▁▁▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁
train/learning_rate,█████▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▁▁▁▁
train/loss,█▆▅▄▂▂▃▁▃▁▂▁▃▁▄▃▃▁▁▁▁▄▃▃▅▂▁▄▄▅▄▃▅▁▃▁▃▄▁▄
total_flos,2.428594759092142e+17
train/epoch,0.50909
train/global_step,350
train/grad_norm,0.04941
train/learning_rate,0
train/loss,0.0453


In [ ]:
model.push_to_hub_merged(
    "matteanedda/mistral-7b-instruct-logical-itinerary-selection-bidirectional-4bit-final",
    tokenizer,
    save_method="merged_4bit_forced",
    token=""
)

Unsloth: Merging 4bit and LoRA weights to 4bit...
This might take 5 minutes...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Done.
Unsloth: Saving 4bit Bitsandbytes model. Please wait...


README.md:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved merged_4bit model to https://huggingface.co/matteanedda/mistral-7b-instruct-logical-itinerary-selection-bidirectional-4bit-final
